# Gemini Pro - Simple RAG with Vertex AI Search as a grounding service.

* This notebook explains how to use grounding service in Gemini Pro.
* **To verify the chunking options of data in a data store** 

In [86]:
%pip install --upgrade --quiet google-cloud-aiplatform
%pip install --upgrade --quiet google-cloud-discoveryengine
from IPython.display import display, Markdown

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [87]:
import os
from google.cloud import storage
from dotenv import load_dotenv, set_key
from google.oauth2 import service_account
import google.oauth2.credentials

# Load Google API Key from .env file
env_path = '.env'
load_dotenv(env_path)
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("Failed to load API key. Please set GOOGLE_API_KEY in the .env file.")

# Service account key file path and project settings
KEY_PATH = "./pablo-test-425702-22d29fa73af8.json"
PROJECT_ID = "pablo-test-425702"
GS_LOCATION = "asia-northeast3"
MODEL_NAME="gemini-1.5-flash"
VERTEX_AI_LOCATION="us-central1"
# Initialize Google Cloud Storage client
# try:
#     storage_client = storage.Client.from_service_account_json(KEY_PATH)
#     print("Google Cloud Storage client initialized")
# except Exception as e:
#     print(f"Google Cloud Storage client error: {e}")

credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH, 
    scopes=['https://www.googleapis.com/auth/cloud-platform']
)




In [88]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part, Tool
import vertexai.generative_models as generative_models

# Grounding service is still in preview.
from vertexai.preview.generative_models import grounding

# Initalizate the current vertex AI execution environment.
vertexai.init(project=PROJECT_ID, location=VERTEX_AI_LOCATION)
# Access to the generative model.
model = GenerativeModel(MODEL_NAME)

In [89]:
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/151473909705/locations/global/collections/default_collection/dataStores/0e234f18-5162-4a39-a978-0a1c99a1e3f0-chunking_1721195392110/servingConfigs/default_search:search"

In [90]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()

In [91]:
import requests
import html

def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{
            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }
      },
  }

  data = json.dumps(query_dic)
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  decoded_text = html.unescape(response.text)
  print(decoded_text)

  
  
  
  return response.text

In [92]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']
            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['content'] = chunk['content']

            # Chunks appear starting from those closest to the current Contents.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

In [93]:
question = "침해사고의 원인 분석에 대해서 설명해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)

{
  "results": [
    {
      "chunk": {
        "name": "projects/151473909705/locations/global/collections/default_collection/dataStores/0e234f18-5162-4a39-a978-0a1c99a1e3f0-chunking_1721195392110/branches/0/documents/00757fab3263dd76a9c501a711c9da67/chunks/c22",
        "id": "c22",
        "content": "④ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 위하여 필요\n하면 정보통신서비스 제공자에게 정보통신망의 접속기록 등 관련 자료의 보전을 명할 수 있다.\n\u003c개정 2013. 3. 23., 2017. 7. 26., 2022. 6. 10.\u003e\n⑤ 과학기술정보통신부장관은 제2항에 따른 침해사고의 원인 분석 및 대책 마련을 하기 위하여\n필요하면 정보통신서비스 제공자에게 침해사고 관련 자료의 제출을 요구할 수 있으며, 중대한\n침해사고의 경우 소속 공무원 또는 제3항에 따른 민ㆍ관합동조사단에게 관계인의 사업장에 출\n입하여 침해사고 원인을 조사하도록 할 수 있다. 다만, 「통신비밀보호법」 제2조제11호에 따른\n통신사실확인자료에 해당하는 자료의 제출은 같은 법으로 정하는 바에 따른다.\u003c개정 2013. 3. 23.,\n2017. 7. 26., 2022. 6. 10.\u003e\n⑥ 과학기술정보통신부장관이나 민ㆍ관합동조사단은 제5항에 따라 제출받은 자료와 조사를 통\n하여 알게 된 정보를 침해사고의 원인 분석 및 대책 마련 외의 목적으로는 사용하지 못하며, 원\n인 분석이 끝난 후에는 즉시 파기하여야 한다.\u003c개정 2013. 3. 23., 2017. 7. 26., 2022. 6. 10.\u003e\n⑦ 제3항에 따른 민ㆍ관합동조사단의 구성ㆍ운영, 제5항에 따라 제출된 자

In [94]:
context = parse_chunks(searched_ctx)

# print(context)

In [95]:
def generate(query:str):
    """
    Generate a response from the model.

    query :
      query to be sent to the model

    Returns:
      The generated response.

    """

    # Set model parameter : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/send-multimodal-prompts#set_model_parameters
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 1,
        "top_p": 0.95,
    }

    # Configure satey setting : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes
    # Refer to the link to remove : https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/configure-safety-attributes#how_to_remove_automated_response_blocking_for_select_safety_attributes
    safety_settings = {
        generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    }

    responses = model.generate_content(
        [query],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    return responses.text

In [96]:
from time import perf_counter

t1_start = perf_counter()

prompt = f"""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}
  """

outcome = generate(prompt)

t1_end  = perf_counter()
print(f"Time : {t1_end - t1_start} seconds\n\n")

display(Markdown(outcome))


Time : 4.716365874977782 seconds




정보통신망 이용촉진 및 정보보호 등에 관한 법률에 따르면, 정보통신서비스 제공자 등 정보통신망을 운영하는 자는 침해사고가 발생하면 침해사고의 원인을 분석하고 그 결과에 따라 피해의 확산 방지를 위하여 사고 대응, 복구 및 재발 방지에 필요한 조치를 하여야 합니다.

과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 침해사고가 발생하면 그 침해사고의 원인을 분석하고 피해 확산 방지, 사고 대응, 복구 및 재발 방지를 위한 대책을 마련하여 해당 정보통신서비스 제공자(공공기관등은 제외한다)에게 필요한 조치를 이행하도록 명령할 수 있습니다.

또한, 과학기술정보통신부장관은 정보통신서비스 제공자의 정보통신망에 중대한 침해사고가 발생한 경우 침해사고의 원인 분석 및 대책 마련을 위하여 필요하면 정보보호에 전문성을 갖춘 민·관합동조사단을 구성하여 그 침해사고의 원인 분석을 할 수 있습니다.

즉, 침해사고 발생 시 정보통신서비스 제공자는 스스로 원인을 분석하고 대응해야 하며, 과학기술정보통신부장관은 침해사고의 원인 분석 및 대책 마련을 위해 명령 또는 조사를 할 수 있습니다. 
